## Oppgave 2 a)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
kb = 1.38 * 10**-23
T = 298
beta = 1/(kb*T)

def V21(x):
    return 1/beta
def V22(x):
    return -x/beta

def walkDirection(pMinus,pPluss):
    tilfeldigVerdi = np.random.ranf()
    if tilfeldigVerdi<=pMinus:
        return -1
    elif tilfeldigVerdi >=(1-pPluss):
        return 1
    else:
        return 0



def sannsynligheter(beta,V,x):
    pMinus = 1/(1+np.exp(-beta*(V(x-1)-V(x+1)))+np.exp(-beta*(V(x)-V(x+1))))
    pPluss = 1/(1+np.exp(-beta*(V(x+1)-V(x-1)))+np.exp(-beta*(V(x)-V(x-1))))
    return pMinus, pPluss


def simulasjon(steg, beta, V, antallPartikler, mellomrom):
    posisjoner = np.arange(100,(antallPartikler)*(mellomrom+1)+100,mellomrom+1)
    print(posisjoner)
    for i in range(steg):
        for j in range(len(posisjoner)):
            pMinus, pPluss = sannsynligheter(beta,V,posisjoner[j])
            skritt =  walkDirection(pMinus,pPluss)
            #if not ((posisjoner[j] + skritt) in posisjoner):
                #posisjoner[i]+=skritt
            posisjoner[j]+=skritt
    return posisjoner

def plotGjennomsnitt(steg, beta, V, antallPartikler, mellomrom, antallSimulasjoner):
    posisjonerArray = np.array([simulasjon(steg, beta, V, antallPartikler, mellomrom) for _ in range(antallSimulasjoner)])
    posisjonerArray.flatten()
    numberArray = np.array([0 for _ in range(200+(antallPartikler-1)*(mellomrom+1)+1)])
    for i in range(len(numberArray)):
        numberArray[i]=posisjonerArray.count(i)
    plt.hist(numberArray)
    plt.show()

    return

print(simulasjon(100,beta,V21,101,2))

[100 103 106 109 112 115 118 121 124 127 130 133 136 139 142 145 148 151
 154 157 160 163 166 169 172 175 178 181 184 187 190 193 196 199 202 205
 208 211 214 217 220 223 226 229 232 235 238 241 244 247 250 253 256 259
 262 265 268 271 274 277 280 283 286 289 292 295 298 301 304 307 310 313
 316 319 322 325 328 331 334 337 340 343 346 349 352 355 358 361 364 367
 370 373 376 379 382 385 388 391 394 397 400]
[105  99  99 107  89 116 109 126 119 130 136 150 119 152 141 155 136 145
 166 157 165 164 180 161 172 175 170 174 184 182 189 202 193 187 197 205
 214 212 203 218 231 224 216 229 220 249 249 231 257 251 230 258 251 263
 267 263 262 268 268 282 275 275 300 284 303 299 296 305 302 295 315 311
 324 322 332 332 310 323 323 332 335 338 346 353 350 344 355 363 368 362
 364 366 369 363 397 380 378 377 413 406 394]
